In [1]:
import mlutils
import warnings
import os
import pandas as pd
warnings.filterwarnings(action="ignore")
os.getcwd()

station = pd.read_csv(
    "assets/seoul_bike_station_01_12.csv", encoding="CP949", index_col=0
)
near_bus = pd.read_csv(
    "assets/near_bus_500m.csv", encoding="CP949", index_col=0
)
seoul_bike = pd.read_parquet(
    "/Users/yangwoolee/git_repo/seoul_bike_dash/parquet/220607_bike_record.parquet.gzip"
)
sub_info = pd.read_csv(
    "assets/sub_and_bike_info.csv", encoding="CP949", index_col=0
)

search_info = pd.read_csv(
    "assets/search_info.csv", encoding="CP949", index_col=0
)
moon_light = mlutils.moon_light(station,near_bus,seoul_bike,sub_info)


In [ ]:
moon_light.route_data(dep_id=4443,arr_id=754)

In [7]:
moon_light.departure_info(3119)

,sub_id,sub_name,line,longtitude,latitude,bike_id
119,4113,당산역,9호선,126.902809,37.533406,"[244, 247, 267, 268, 272, 290, 3201]"


> ### 디버그 용도

In [3]:
#  def route_data(dep_id, arr_id):

dep_id = 4443
arr_id = 754
dep_info = moon_light.departure_info(dep_id)  # dataframe
arr_info = moon_light.arrival_info(arr_id)  # dataframe

if dep_id < 3000:
    print("자전거")
    bike_coor = moon_light.route_coor(dep_info, arr_info)
    retu = dict(bike_coor=bike_coor)

elif dep_id > 4000:
    print("버스")
    # * -- 자전거 대여소 정보 추출 -- *

    # * -- 출도착 버스 정보-- *
    dep_arr_bus_info = moon_light.finding_start_end(dep_info, arr_info)

    # * -- 경로 선택 및 경로 노선 추출 -- *
    bus_whole, waypoint, bus_start_end = moon_light.bus_route_info(dep_arr_bus_info)

    # * -- 좌표 받기 -- *


    arr_trans = bus_start_end.iloc[[1]]

    dep_bike_info = moon_light.near_bus_bike_info(
    bus_start_end.iloc[[0]], arr_info
    ).sort_values(by="num", ascending=False)

    arr_bike_info = moon_light.near_bus_bike_info(
    arr_trans, arr_info
    ).sort_values(by="num", ascending=False)

    dep_arr_bike = moon_light.route_coor(dep_bike_info, arr_info)


    bus_coor = moon_light.bus_route_coor(bus_start_end, waypoint)
    walk_coor = moon_light.route_coor(
        arr_trans, arr_bike_info.iloc[[0]]
    )
    bike_coor = moon_light.route_coor(arr_bike_info.iloc[[0]], arr_info)
    retu= dict(
        dep_arr_bike=dep_arr_bike,
        bus_coor=bus_coor,
        walk_coor=walk_coor,
        bike_coor=bike_coor,
    )
else:
    print("지하철")
    # * -- 자전거 대여소 정보 추출 -- *
    near_bus_bike_info = moon_light.near_bus_bike_info(
        dep_info, arr_info
    ).sort_values(by="dist")
    departure_bike_station = pd.DataFrame(near_bus_bike_info.iloc[0]).T

    # * -- 자전거 경로(사실 보행로 추천이라는 사실~)-- *
    walk_coor = moon_light.route_coor(dep_info, departure_bike_station)
    bike_coor = moon_light.route_coor(departure_bike_station, arr_info)
    retu= dict(
        walk_coor=walk_coor, bike_coor=bike_coor
    )

버스


In [47]:
import ast
import numpy as np
## station_to_station 구상
# 여의도 환승센터 인근 500m 따릉이 대여소 검색 & 50건 이상 정보만 반환하기 
# 목적지 대여소와 관련 정보가 있는지 확인
# 정보가 있다면 bike 경로도 제시
# 정류장 & 역 departure => 자전거 departure
# 자전거 departure => 자전거 arrival
# 정보가 없다면 관련 정보가 없다고 팝업뜨고 다시 선택해달라고 하기


bike_id = ast.literal_eval(departure['bike_id'].iloc[0])

raw = moon_light.raw_data(arrival_info.index[0])


br_record = raw.groupby('st_id1').size()
print(br_record)
br_record_id = br_record[br_record >50].index
print(br_record_id)

result = np.intersect1d(bike_id,br_record_id).tolist()
print(result)



[700,773,1169,1171,2703,3761,5067]
st_id1
102     1
103     3
104     6
105     2
106     5
       ..
4582    1
4584    1
4605    1
4625    2
9999    1
Length: 400, dtype: int64
Int64Index([ 272,  700,  702,  703,  719,  723,  726,  731,  735,  744,  746,
             747,  749,  752,  754,  755,  756,  758,  764,  765,  766,  768,
             769,  770,  771,  773,  776,  786,  792,  793, 1115, 1169, 3763,
            4526, 4527],
           dtype='int64', name='st_id1')
[700, 773, 1169]


In [ ]:
# search box info

sub_info = pd.read_csv('assets/sub_and_bike_info.csv',encoding='CP949',index_col=0)
bus_info = pd.read_csv(
            "assets/near_bus_500m.csv", encoding="CP949", index_col=0
        )
bike_info = pd.read_csv(
            "assets/seoul_bike_station_01_12.csv", encoding="CP949", index_col=0
        )

## search_box 정보
d = pd.read_csv('assets/search_info.csv',encoding='CP949',index_col=0)
a = d[d['name'].str.contains('당산')].drop_duplicates('name')

# id에 맞는 info 불러오기
b = a['id'].iloc[-1]
if b < 3000 : 
    departure_info = bike_info.iloc[b]
elif b >4000 : 
    departure_info = bus_info.iloc[b-4000]
else :
    departure_info = sub_info.iloc[b-3000]

print(departure_info)
# departure_info = moon_light.departure_info('당산') 
arrival_info = moon_light.arrivial_info(754)
# route_info=moon_light.finding_start_end(departure_info, arrival_info)


bike_id       [230,240,264,265,266,3206,3221]
bus                                       N64
order                                      34
id                                      19163
name                                 당산동진로아파트
longtitude                         126.896991
latitude                            37.521002
Name: 1449, dtype: object


> ### 이름으로 버스 정류장과 대여소 선택

In [ ]:
import plotly.graph_objects as go
import plotly.express as px

#-- 자전거 시작 대여소 -- #
# dataframe밖에 안되는듯.
fig = px.scatter_mapbox(pd.DataFrame(departure_station).T, lat='latitude', lon="longtitude")
fig.update_traces(
    marker=go.scattermapbox.Marker(symbol="circle", size=10, opacity=0.7, color="blue")
)

# -- bus 정류장 출발 도착 지점 --#
fig_2 = px.scatter_mapbox(route_start_and_end, lat="latitude", lon="longtitude", hover_name="name")
fig_2.update_traces(
    marker=go.scattermapbox.Marker(symbol="circle", size=10, opacity=0.7, color="#AD00FF")
)

# --반경 내 따릉이 대여소--#
fig_3 = go.Figure(
    go.Scattermapbox(
        name=f"station with in {round(500/1000,2)}km",
        lat=bike_recommend["latitude"],
        lon=bike_recommend["longtitude"],
        mode="markers",
        marker=go.scattermapbox.Marker(size=8, color="black", opacity=0.2),
        text=bike_recommend["st_name"],
    )
)

# --목적지 대여소 -- #
fig_4 = go.Figure(
    go.Scattermapbox(
        name="location",
        lat=arrival_info["latitude"],
        lon=arrival_info["longtitude"],
        mode="markers",
        # marker = {'size': 20, 'symbol': "bus"},
        marker=go.scattermapbox.Marker(size=10, color="fuchsia", opacity=0.8),
        text=arrival_info["st_name"].values,
    )
)

# -- 버스 route -- #
fig_5 = go.Figure(
    go.Scattermapbox(
        lat=bus_route_coor[1],
        lon=bus_route_coor[0],
        mode="lines",
        marker=dict(symbol="circle", size=15, color='#AD00FF' , opacity=0.1),
    )
)

# -- 도보 route -- #
fig_6 = go.Figure(
    go.Scattermapbox(
        lat=walk_route_coor[1],
        lon=walk_route_coor[0],
        mode="lines",
        marker=dict(symbol="circle", size=15, color='green' , opacity=0.1),
    )
)

# -- 자전거 route -- #
fig_7 = go.Figure(
    go.Scattermapbox(
        lat=bike_route_coor[1],
        lon=bike_route_coor[0],
        mode="lines",
        marker=dict(symbol="circle", size=15, color='black' , opacity=0.5),
    )
)




# -- main route에 종합 -- #
fig.add_trace(fig_2.data[0])
fig.add_trace(fig_3.data[0])
fig.add_trace(fig_4.data[0])
fig.add_trace(fig_5.data[0])
fig.add_trace(fig_6.data[0])
fig.add_trace(fig_7.data[0])

fig.update_layout(
    mapbox=dict(
        accesstoken="pk.eyJ1IjoieWFuZ29vcyIsImEiOiJjbDNqd2tkN2IwbGdmM2pvNzF0c2M4NnZkIn0.J3IjPYg3w28cGiWkUD7bnA",
        # style='mapbox://styles/yangoos/cl3jubvl7000c14llgtoev0nm'
    ),
    margin=dict(l=20, r=20, t=20, b=20)
)
fig.show()


In [ ]:
ending_point  = station.query("id == @ending_bike_id")[['경도','위도']].values[0]

starting_point = result_recommend[['경도','위도']].values[0]


> ### 길찾기 기능(비추..)

In [ ]:
a = [["합정역", 126.913997,  37.549682] 
,["당산역", 126.902410, 37.534863]]

In [ ]:
import requests


def bike_route(departure, arrival) :
    url = "https://apis.openapi.sk.com/tmap/routes/pedestrian?version=1"

    payload = {
        "angle": 0,
        "speed": 0,
        "reqCoordType": "WGS84GEO",
        "searchOption": "0",
        "resCoordType": "WGS84GEO",
        "sort": "index",
        "startX": a[0][1],
        "startY": a[0][2],
        "endX": a[1][1],
        "endY": a[1][2],
        "startName": a[0][0],
        "endName": a[0][0],
    }
    headers = {
        "Accept": "application/json",
        "Content-Type": "application/json",
        "appKey": "l7xxfdc75c1509a74ecdba02bf5e024ee9d5"
    }

    response = requests.post(url, json=payload, headers=headers).text

    api_data = json.loads(response).get('features')

    coor_raw_data = [api_data[i].get('geometry').get('coordinates') for i in range(0,len(api_data))]

    finish_data = []
    for data in coor_raw_data :
        # data = api_data[1].get('geometry').get('coordinates')
        if type(data[0]) == list :
            for i in data:
                finish_data.append(i)
        else :
            pass
    return pd.DataFrame(finish_data).T.values



In [ ]:
b = pd.DataFrame(finish_data).T.values

fig = go.Figure(
    go.Scattermapbox(
        lat=b[1],
        lon=b[0],
        mode="lines",
        marker=dict(symbol="circle", size=15, color='#black' , opacity=0.5),
    )
)
# fig.add_trace(fig_1.data[0])

fig.update_layout(
    mapbox=dict(
        accesstoken="pk.eyJ1IjoieWFuZ29vcyIsImEiOiJjbDNqd2tkN2IwbGdmM2pvNzF0c2M4NnZkIn0.J3IjPYg3w28cGiWkUD7bnA",
        # style='mapbox://styles/yangoos/cl3jubvl7000c14llgtoev0nm'
    ),
    margin=dict(l=20, r=20, t=20, b=20)
)
fig.show()


> ### 자전거 대여소 간 계산
* 해당 경우 검색 시 
* 지하철 -> 대여소 -> 대여소
* 대여소 -> 대여소 

In [ ]:


# 345ms
def st_to_st_new (starting_station, ending_station):

    lat_1=station.query('id==@starting_station')['위도'].values
    lon_1=station.query('id==@starting_station')['경도'].values
    lat_2=station.query('id==@ending_station')['위도'].values
    lon_2=station.query('id==@ending_station')['경도'].values
    dist_station = haversine_np(lat_1,lon_1,lat_2,lon_2)

    BM_1 = (seoul_bike['st_id1'] == starting_station) & (seoul_bike['st_id2'] == ending_station)
    BM_2 = (seoul_bike['st_id2'] == starting_station) & (seoul_bike['st_id1'] == ending_station)

    # 평균거리 계산
    sort_station = seoul_bike[BM_1 | BM_2]
    mean_station = sort_station[sort_station['dist'] < dist_station[0]*2]['riding_time'].value_counts()[:5].index.values.mean()
    num_station = len(sort_station)

    print(mean_station,num_station,f'{dist_station[0]/1000 :.2f}km' )

st_to_st_new(207, 754)


In [ ]:
## 옛날자료

def st_to_st(starting_station, ending_station):

    """
    starting_station = list or val이고 bike_station id가 들어감
    ending_station = list or val이고 bike_station id가 들어감
    """

    # int 또는 list로 starting station 받기
    if type(starting_station) == int:
        starting_station_list = list()
        starting_station_list.append(starting_station)
    else:
        starting_station_list = starting_station

    for starting_station in starting_station_list:
        BM_1 = (seoul_bike_sorting["st_id1"] == starting_station) & (
            seoul_bike_sorting["st_id2"] == ending_station
        )
        BM_2 = (seoul_bike_sorting["st_id2"] == starting_station) & (
            seoul_bike_sorting["st_id1"] == ending_station
        )

        # 평균거리 계산
        mean_station = (
            seoul_bike_sorting[BM_1 | BM_2]["riding_time"]
            .value_counts()
            .sort_values(ascending=False)[:5]
            .index.values.mean()
        )

        # 이용기록 계산
        num_station = len(seoul_bike_sorting[BM_1 | BM_2])

        # 직선거리 계산
        lat_1 = station.query("id==@starting_station")["위도"].values
        lon_1 = station.query("id==@starting_station")["경도"].values
        lat_2 = station.query("id==@ending_station")["위도"].values
        lon_2 = station.query("id==@ending_station")["경도"].values
        dist_station = haversine_np(lat_1, lon_1, lat_2, lon_2)
        print(mean_station, num_station, f"{dist_station[0]/1000 :.2f}km")
    # return


st_to_st(counts_station, 754)
